In [2]:
# import os, sys
# from google.colab import drive
# drive.mount('/content/drive')
# my_path = '/content/notebooks'

In [1]:
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from konlpy.tag import Okt
import re
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

okt = Okt()

D:\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
test_data = pd.read_csv('test_data.csv' , encoding = 'euc-kr')
train_data = pd.read_csv('train_data.csv' , encoding = 'euc-kr')

In [3]:
import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)
import konlpy
print(konlpy.__version__)
print(pd.__version__)
print(np.__version__)

2.12.0
2.12.0
0.6.0
1.4.4
1.23.5


In [4]:
# loaded_model = load_model('/content/drive/MyDrive/data_review/best_model.h5')
# print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

# 버전 확인
# pip install --upgrade tensorflow==2.12.0
# pip install --upgrade keras==2.12.0

loaded_model = load_model('best_model2.h5')

In [5]:
# 불용어를 지정하여 필요없는 토큰들은 제거하도록 합니다.

stopwords = ['도', '는', '다', '의', '가', '이', '은', 
             '한', '에', '하', '고', '을', '를', '인', '듯', 
             '과', '와', '네', '들', '듯', '지', '임', '게']

In [6]:
# 단어 집합(vocabulary)의 크기 : 64152
# 등장 빈도가 1번 이하인 희귀 단어의 수: 29480
# 단어 집합에서 희귀 단어의 비율: 45.953360768175585
# 전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.289171484296422

In [7]:
# total_cnt = 64152
# rare_cnt = 29480
# vocab_size = total_cnt - rare_cnt + 2
# tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 

In [8]:
# 긍정 리뷰보다는 부정 리뷰가 좀 더 길게 작성된 경향이 있는 것 같습니다.

X_train = train_data['tokenized'].values
y_train = train_data['label'].values
X_test= test_data['tokenized'].values
y_test = test_data['label'].values

In [9]:
# 정수 인코딩

# 기계가 텍스트를 숫자로 처리할 수 있도록 훈련 데이터와 테스트 데이터에 정수 인코딩을 수행해야 합니다. 
# 훈련 데이터에 대해서 단어 집합(vocaburary)을 만들어봅시다.

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)


In [10]:
total_cnt = 64152
rare_cnt = 29480
vocab_size = total_cnt - rare_cnt + 2

In [11]:
# 단어 집합의 크기는 34674개입니다. 
# 이를 토크나이저의 인자로 넘겨주고, 텍스트 시퀀스를 정수 시퀀스로 변환합니다. 
# 정수 인코딩 과정에서 이보다 큰 숫자가 부여된 단어들은 OOV로 변환하겠습니다.

tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [12]:
max_len = 330

In [13]:
# 훈련용 리뷰의 99.99%가 330 이하의 길이를 가집니다. 훈련용 리뷰를 길이 330으로 패딩하겠습니다.

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)


In [14]:
# # test파일 업로드

# from google.colab import files

# # 파일 선택 대화상자 표시
# uploaded = files.upload()

# # 업로드한 파일 확인
# for filename in uploaded.keys():
#     print(f'Uploaded file: {filename}')


In [21]:
# test 파일 업로드 2
#uploaded
# test_re = pd.read_csv('/content/drive/MyDrive/(주)시더스초밥제주연동점_review.csv' , encoding = 'euc-kr')
test_re = pd.read_csv('(주)시더스초밥제주연동점_review.csv' , encoding = 'euc-kr')
test_re = pd.read_excel('(주)서촌제 스시100_review.xlsx')


In [22]:
# 결측치 처리
test_re = test_re.dropna().reset_index(drop=True)

In [17]:
def sentiment_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
    new_sentence = okt.morphs(new_sentence)
    new_sentence = [word for word in new_sentence if not word in stopwords]
    encoded = tokenizer.texts_to_sequences([new_sentence])
    pad_new = pad_sequences(encoded, maxlen = max_len)

    score = float(loaded_model.predict(pad_new))
    if (score > 0.5):
        print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
    else :
        print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))


In [23]:
# test
for num , i in enumerate(test_re.리뷰.tolist()) :
    print(num)
    print(i)
    print(sentiment_predict(i))
    print()

0
가격이 올라서 슬프지만 맛있어요
1/1 [==============================] - 0s 25ms/step
85.26% 확률로 긍정 리뷰입니다.
None

1
가끔 가는데 갈때마다 맛있어서 갔어요 ㅎㅎ 안경끼신분이 사장님인지 매니저님인지 되게 친절하세요 목소리도 우렁 차시고 ㅎㅎ 근데 오늘 한치가 너무 질겼어요 매운소고기초밥 생겼길래 시켰는데 너무 안익혀서 주셔서 질겨서 뱉었어요 ㅠㅠㅠ 오늘은 조금 속상했지만 담에는 다시 맛있길 ~~
1/1 [==============================] - 0s 26ms/step
89.72% 확률로 부정 리뷰입니다.
None

2
매장 찾기가 좀 어려웠는데 신선한 재료와 친절함이 만족합니다
1/1 [==============================] - 0s 24ms/step
84.35% 확률로 긍정 리뷰입니다.
None

3
공항가기전에 드른건데.입장순서가아닌.좌석나는순서대로 앉을수있어서 웨이팅이좀길거빼고는.음식도맛있고.직원분들이친절하셔서 좋아요~^^
1/1 [==============================] - 0s 26ms/step
52.25% 확률로 부정 리뷰입니다.
None

4
스시 맛은 그냥 그렇고요.. 이렇게 시큼한 락교는 처음 먹어보네요;; 시큼하고 물렁한 락교라니.. 스시 먹을 때 락교와 초생강 굉장히 중요하게 생각해서 다시는 여기 안올것같아요
1/1 [==============================] - 0s 26ms/step
71.84% 확률로 부정 리뷰입니다.
None

5
😊친절하시고 음식도 맛있어요!
1/1 [==============================] - 0s 28ms/step
84.22% 확률로 긍정 리뷰입니다.
None

6
좋아요
1/1 [==============================] - 0s 27ms/step
78.74% 확률로 긍정 리뷰입니다.
None

7
음식이 괜찮았어요
1/1 [================

1/1 [==============================] - 0s 26ms/step
84.22% 확률로 긍정 리뷰입니다.
None

57
오우 맛있네요
1/1 [==============================] - 0s 28ms/step
82.25% 확률로 긍정 리뷰입니다.
None

58
제주 갈때 마다… 가성비 짱짱 수목원도 구경하공
1/1 [==============================] - 0s 25ms/step
81.90% 확률로 긍정 리뷰입니다.
None

59
아이들이 골라먹는 재미에 좋아해서 가끔 가는곳입니다..
매장 넓고 골라먹는 재미로 가는 곳이예요~~^^

한라수목원이 있어 볼거리도 많아서.
밥 먹고 산책하기 좋은 곳입니다^^
1/1 [==============================] - 0s 26ms/step
73.86% 확률로 부정 리뷰입니다.
None

60
오랜만에 갔는데 가격은 올랐는데 퀄리티는 그다지.. 
초밥 만드시는 분들은 설명도 잘해주시고 친절했어요.
1/1 [==============================] - 0s 27ms/step
73.64% 확률로 긍정 리뷰입니다.
None

61
맛도 좋고 친절하기도해서 자주갔던곳이지만 이번에 개빡쳐서 두번다시안가게됨. 원래 레일에 없으면 주문해서 시켜먹는거아닌가요? 한두세번 주문하니까 거기서 초밥만드는 아저씨가 여기가 오마카세냐며 뭐라하는거임.아니 무한리필 초밥집도 아니고 내돈 내고 내가먹는데 기분 팍상해서 밥맛다떨어져 기분나빠서 바로나옴,살다살다 회전초밥집에서 주문하는게 이따위말을 들어야되는지 싶어서 리뷰남김 너무빡쳐 두번다시 절대로안감.지인들이 간다고해도 뜯어말릴거임
돈내고 먹으면서 그딴말들으니 기분이 그지같음
1/1 [==============================] - 0s 26ms/step
92.45% 확률로 부정 리뷰입니다.
None

62
100가지 스시와 음식을 먹을 수 있는
서촌제스시100
미니우동이나 소바도 맛나고
멘

1/1 [==============================] - 0s 25ms/step
92.50% 확률로 부정 리뷰입니다.
None

111
맛있어요!
1/1 [==============================] - 0s 26ms/step
78.74% 확률로 긍정 리뷰입니다.
None

112
좋아요
1/1 [==============================] - 0s 27ms/step
78.74% 확률로 긍정 리뷰입니다.
None

113
굿
1/1 [==============================] - 0s 25ms/step
78.74% 확률로 긍정 리뷰입니다.
None

114
하구한날 가기에는 조금 부담스러운 가격이지만, 맛은 좋다.
1/1 [==============================] - 0s 26ms/step
81.90% 확률로 긍정 리뷰입니다.
None

115
음식맛있고 좋아요~^^
1/1 [==============================] - 0s 26ms/step
84.22% 확률로 긍정 리뷰입니다.
None

116
제주 갈때마다 들려요.
가격이 살곰살곰 올라가지만
광어묵은지.가지.참치.젤 맛있어요.
1/1 [==============================] - 0s 25ms/step
73.64% 확률로 긍정 리뷰입니다.
None

117
분위기에 취하는곳...
1/1 [==============================] - 0s 26ms/step
84.22% 확률로 긍정 리뷰입니다.
None

118
빨리가야해요 
웨이팅 있어요 
너무 괜찮아요 여기
1/1 [==============================] - 0s 27ms/step
83.27% 확률로 긍정 리뷰입니다.
None

119
잘먹고 갑니다 나쁘지 않아요 파인애플이랑 매론이 몇바퀴씩 도는게 좀 아쉽고 (촉촉해 보이지 않음) 음료수 종류가 적어용 닥터페퍼ㅜ먹고싶다
1/1 [========================

1/1 [==============================] - 0s 26ms/step
83.27% 확률로 긍정 리뷰입니다.
None

178
이곳은 불친절하다고 느껴졌어요. 기분좋게 외식하러 갔다가 여자직원분 응대가 너무 기분나빳어요. 회전초밥집은 소바나 사이다 주문하면 안되나요? 정말 먾이 귀찮아 하는 모습이 심하게 느껴졌네요. 주위에 추천드리고 싶지 않은 가게입니다.
1/1 [==============================] - 0s 28ms/step
85.11% 확률로 부정 리뷰입니다.
None

179
맛있어요~~
1/1 [==============================] - 0s 24ms/step
78.74% 확률로 긍정 리뷰입니다.
None

180
가격이 올랐어요
1/1 [==============================] - 0s 27ms/step
82.25% 확률로 긍정 리뷰입니다.
None

181
맛보다는 조용히 얘기하며 천천히 먹을수 있는 장소에요
1/1 [==============================] - 0s 26ms/step
81.90% 확률로 긍정 리뷰입니다.
None

182
서촌제 올해 초반 가격은 1700/2500원이었어요 1700원도 먹을 것도 많고 연어 광어 포함이었지요
굉장히 자주 가던 / 최애 / 가성비 갑의 스시집이었습니다 
6월14일 방문 때 가격이 올랏더라고요 1900 / 2900원으로 올랐었어요 그 당시 연어 가격 폭등 / 물가 오름으로 가격 올리는건 이해할수 있는 정도수준이었지요(이때부터 먹을만한건 거의 상당 수 2900원짜리였어요) 어느곳이든 안오른곳이 없어서 괜찮았어요
그런데 22일만에 다시 방문을 했는데 1900 / 2300/2900원으로 3단계로 나뉘어졌어요 1900원은 계란과 가지뿐이고 2300원(소라 한치 등)이 되고 광어 연어 등은 2900원이 되었어요 예전에는 그가격에 퀄리티가좋은거였는데 이가격에 이정도는..
두명이가면 기본8-9마넌이예요
1/1 [=======

1/1 [==============================] - 0s 27ms/step
85.26% 확률로 긍정 리뷰입니다.
None

244
제주사는 친구와 방문했어요. 회전스시 좋아하신다면 여기추천요~~^^
1/1 [==============================] - 0s 26ms/step
80.25% 확률로 긍정 리뷰입니다.
None

245
딱 먹을것만 있는것 같아요. 나무접시와 일반접시가 가격차가 있는데, 나무접시에 있는것들이 주로 먹을만 했어요. 배고파서 많이 먹긴 했지만, 엄청난 금액에 깜놀했네요. 묵은지광어스시가 개운하고 맛났어요.
1/1 [==============================] - 0s 25ms/step
78.83% 확률로 부정 리뷰입니다.
None

246
굿
1/1 [==============================] - 0s 27ms/step
78.74% 확률로 긍정 리뷰입니다.
None

247
좋아요
1/1 [==============================] - 0s 26ms/step
78.74% 확률로 긍정 리뷰입니다.
None

248
굳
1/1 [==============================] - 0s 26ms/step
78.74% 확률로 긍정 리뷰입니다.
None

249
맛있어요
1/1 [==============================] - 0s 26ms/step
78.74% 확률로 긍정 리뷰입니다.
None

250
맛있고 저렴합니다!
1/1 [==============================] - 0s 25ms/step
82.25% 확률로 긍정 리뷰입니다.
None

251
스시도크고 맛있어요
가격올라서 아숩
1/1 [==============================] - 0s 27ms/step
85.58% 확률로 긍정 리뷰입니다.
None

252
좋아요!
1/1 [==============================] - 0s 27ms/step
7

1/1 [==============================] - 0s 28ms/step
82.25% 확률로 긍정 리뷰입니다.
None

308
골라먹는 재미에 다양하게 구성되어 있어서 잘 먹고 갑니다~~
1/1 [==============================] - 0s 26ms/step
81.90% 확률로 긍정 리뷰입니다.
None

309
맛도 좋고 가격도 저렴하고...
초밥 좋아하는 딸아이덕에 3박4일 일정 중 2번이나 방문한 곳입니다.ㅎ
1/1 [==============================] - 0s 25ms/step
64.81% 확률로 긍정 리뷰입니다.
None

310
신선하고 가격도 좋아요! 종류도 많아 먹기 좋습니다! 사람도 많아서 항상 웨이팅 20-30분정도는 기본입니다. 근데 유독 홀에있는 키큰 남자 안경쓰신분은 말투가 항상 툭툭 불친절하신듯 해요 ..
1/1 [==============================] - 0s 26ms/step
77.34% 확률로 부정 리뷰입니다.
None

311
스시가 생각날때 가족들과 자주 가는 곳이에요~ 가성비도 좋고, 매장도 넓고 깨끗해요~~^^
1/1 [==============================] - 0s 27ms/step
73.64% 확률로 긍정 리뷰입니다.
None

312
모든 초밥에 와사비를 넣지 않아 아이들과 가서 먹기 아주 좋네요. 스시도 대체로 퀄러티가 괜찮고 롤도 맛있어요
1/1 [==============================] - 0s 26ms/step
67.96% 확률로 긍정 리뷰입니다.
None

313
맛있습니다 친절합니다
1/1 [==============================] - 0s 27ms/step
82.25% 확률로 긍정 리뷰입니다.
None

314
줄서서먹는 회전초밥집!! 맛있어요 가격도 적당하고 !!
1/1 [==============================] - 0s 27ms/step
85.13% 확률로

1/1 [==============================] - 0s 27ms/step
58.74% 확률로 부정 리뷰입니다.
None

372
저렴한 가격에 맛도 좋아요 😀 주차하기도 편하고 주변에 볼거리도 많습니당!!
1/1 [==============================] - 0s 28ms/step
78.32% 확률로 긍정 리뷰입니다.
None

373
먹고싶은것만 골라먹을수있고 초밥가격도 좋아서 맘편히 먹고왔네요
1/1 [==============================] - 0s 26ms/step
80.25% 확률로 긍정 리뷰입니다.
None

374
시작부터 끝까지 별로여서 만족한 식사는 못했다. 직원분들 친절함에 중간점수 누름.
1.예써어플 줄서기 갑자기 중단함(민원때문)ㅡ이해함. 2.대기16번에서 대기1번되어 기다리는데ㅡ갑자기 대기2번으로 바뀜. 이유인즉슨 번호 지나간 사람이 갑자기 나타났다함. 그건 거기사정이고ㅡ 그럼 맨뒤로 넘겨야지. 왜 내 앞에?? 진짜 화났음 3.다먹은 그릇 10개초과로 계산. 이건 진짜 손님이 잘 체크해야함. 세기쉽도록 20개단위로 쌓아올림. 직원분 셀 때마다 44개 36개ㅋㅋ 결국 34개로 내가 체크함ㅡ 4.막판되니 새우.연어만 보임. 초밥만드시는 분이 필요한거 주문하라하시고 매우 친절하시나ㅡ눈에 음식이 안보이니 더 이상 안들어감. 그렇다고 계속 만들어 놓지지는 못할거구ㅡ 기대가 많아서인지 실망이 컸음
1/1 [==============================] - 0s 28ms/step
92.50% 확률로 부정 리뷰입니다.
None

375
좋아요
1/1 [==============================] - 0s 27ms/step
78.74% 확률로 긍정 리뷰입니다.
None

376
가성비 있네요
잘 먹었어요!!
1/1 [==============================] - 0s 26ms/step
85.65% 확률로 긍정 리뷰입니다.
None

377
아마도 우리나라 브렌드중 

1/1 [==============================] - 0s 26ms/step
85.26% 확률로 긍정 리뷰입니다.
None

423
바쁘신중에도 항상 친절하게 음식내주시는 쉐프님들 참 인상 깊었어요 맛있게 잘 먹고갑니다 주위에 소개 많이 할게요^^
1/1 [==============================] - 0s 26ms/step
54.63% 확률로 긍정 리뷰입니다.
None

424
가격 퀄리티 둘다 만족스럽다는 소문듣고 방문했는데 인정인정~ 
손님이 많아 잠시 웨이팅하고 식사했어요
그만큼 회전률이 좋아 신선한 스시 많이 먹었습니다
1/1 [==============================] - 0s 27ms/step
61.76% 확률로 부정 리뷰입니다.
None

425
또 올께요~ 더이상 긴말 필요없는 곳
1/1 [==============================] - 0s 26ms/step
85.13% 확률로 긍정 리뷰입니다.
None

426
합리적인 가격에 
신선하고 맛있는 초밥을 먹었어요
1/1 [==============================] - 0s 26ms/step
85.13% 확률로 긍정 리뷰입니다.
None

427
굿
1/1 [==============================] - 0s 25ms/step
78.74% 확률로 긍정 리뷰입니다.
None

428
맛있어요.친절해요 .싱싱해요
1/1 [==============================] - 0s 27ms/step
84.22% 확률로 긍정 리뷰입니다.
None

429
직원분들도 친절하시고 이렇게 저렴한 회전초밥은 의심? 해보는편인데 ㅎㅎㅎㅎㅎㅎ 맛도 괜찮고 매장도 청결하고 배부르게 잘 먹고 왔습니다 :-)
원하시는 메뉴가 있으시면 요청하면 만들어 주시기도 하구요 마지막에 파인애플까지 ㅋㅋ 완벽했어요 너무너무 잘 먹었어요 👍🏻
1/1 [==============================] - 0s 25ms/step
83.